In [ ]:
# hide
# default_exp datasets

from nbdev.showdoc import *

# Data Preparation

Whether working with simulated data or the real deal, it has to be represented in a standardized way to enable interchangeable use of library functions.

In general, we prepare data in two formats:

- **Psifr-Format**. We use or modify code from the Psifr library for analysis of free recall data. Documentation of the data format expected by the library is provided in detail [here](https://psifr.readthedocs.io/en/latest/guide/import.html). In particular, our data preparation code tends to import data as specified in the linked documentation and then apply `psifr.merge_free_recall` to the imported DataFrame to enable direct computation of key summary statistics using other functions in the library.

- **Trials Array**. The dataframes used by Psifr to prepare visualizes are useful for that purpose, but aren't ideal for the numerical calculations that often underlie tasks like model fitting. In prepared `trials` arrays, each row corresponds to a unique trial that some subject performed for the study, while each entry corresponds to a recall event. Each positive integer refers to a unique item within the context of that trial, while 0 can be safely coded as recall termination, and negative integers track unique intrusions (though we normally exclude those during data preparation).

Our data preparation functions also tend to output associated metadata - e.g. list lengths, subject indices, condition vectors, and so on - useful for ensuring the correctness or speed of functions using them, or just for selecting subsets of the dataset for individualized analysis. I tend to add these in a haphazard way - when I realize I might use them! By including them in a dataset's associated preparation function, I avoid having to reimplement an extraction routine myself later on.

## Murdock1962 Dataset
This dataset doesn't have any item repetitions, but could be useful for sanity checks and whatnot.

Our data structure associated with Murdock (1962) has three `LL` structures that each seem to correspond to a different data set with different list lengths.  Inside
each structure is:
- `recalls` with 1200 rows and 50 columns. Each row presumably represents a subject, and each column seems to
  correspond to a recall position, with -1 coded for intrusions. `MurdData_clean.mat` probably doesn't have these
  intrusions coded at all.
- `listlength` is an integer indicating how long the studied list is.
- `subject` is a 1200x1 vector coding the identities of each subject for each row. Each subject seems to get 80 rows a
  piece. He really got that much data for each subject?
- `session` similarly codes the index of the session under consideration, and it's always 1 in this case.
- `presitemnumbers` probably codes the number associated with each item. Is just its presentation index.

We'll enable selection of relevant information from these structures based on which `LL` structure we're interested in using a `dataset_index` parameter.

In [ ]:
# export
# hide
import scipy.io as sio
import numpy as np
import pandas as pd
from psifr import fr

def prepare_murddata(path, dataset_index):
    """
    Prepares data formatted like `data/MurdData_clean.mat` for fitting.

    Loads data from `path` with same format as `data/MurdData_clean.mat` and
    returns a selected dataset as an array of unique recall trials and a
    dataframe of unique study and recall events organized according to `psifr`
    specifications.

    **Arguments**:
    - path: source of data file
    - dataset_index: index of the dataset to be extracted from the file

    **Returns**:
    - trials: int64-array where rows identify a unique trial of responses and
        columns corresponds to a unique recall index.
    - merged: as a long format table where each row describes one study or
        recall event.
    - list_length: length of lists studied in the considered dataset
    """
    # load all the data
    matfile = sio.loadmat(path, squeeze_me=True)
    murd_data = [matfile['data'].item()[0][i].item() for i in range(3)]

    # encode dataset into psifr format
    trials, list_length, subjects = murd_data[dataset_index][:3]
    trials = trials.astype('int64')

    data = []
    for trial_index, trial in enumerate(trials):

        # every time the subject changes, reset list_index
        if not data or data[-1][0] != subjects[trial_index]:
            list_index = 0
        list_index += 1

        # add study events
        for i in range(list_length):
            data += [[subjects[trial_index],
                      list_index, 'study', i+1, i+1]]

        # add recall events
        for recall_index, recall_event in enumerate(trial):
            if recall_event != 0:
                data += [[subjects[trial_index], list_index,
                          'recall', recall_index+1, recall_event]]

    data = pd.DataFrame(data, columns=[
        'subject', 'list', 'trial_type', 'position', 'item'])
    merged = fr.merge_free_recall(data)
    return trials, merged, list_length

In [ ]:
murd_trials, murd_events, murd_length = prepare_murddata(
    '../../data/MurdData_clean.mat', 0)

murd_events.head()

## Lohnas2014 Dataset
> Siegel, L. L., & Kahana, M. J. (2014). A retrieved context account of spacing and repetition effects in free recall. Journal of Experimental Psychology: Learning, Memory, and Cognition, 40(3), 755.

Across 4 sessions, 35 subjects performed delayed free recall of 48 lists. Subjects were University of Pennsylvania undergraduates, graduates and staff, age 18-32. List items were drawn from a pool of 1638 words taken from the University of South Florida free association norms (Nelson, McEvoy, & Schreiber, 2004; Steyvers, Shiffrin, & Nelson, 2004, available at http://memory.psych.upenn.edu/files/wordpools/PEERS_wordpool.zip). Within each session, words were drawn without replacement. Words could repeat across sessions so long as they did not repeat in two successive sessions. Words were also selected to ensure that no strong semantic associates co-occurred in a given list (i.e., the semantic relatedness between any two words on a given list, as determined using WAS (Steyvers et al., 2004), did not exceed a threshold value of 0.55).

Subjects encountered four different types of lists: 
1. Control lists that contained all once-presented items;  
2. pure massed lists containing all twice-presented items; 
3. pure spaced lists consisting of items presented twice at lags 1-8, where lag is defined as the number of intervening items between a repeated item's presentations; 
4. mixed lists consisting of once presented, massed and spaced items. Within each session, subjects encountered three lists of each of these four types. 

In each list there were 40 presentation positions, such that in the control lists each position was occupied by a unique list item, and in the pure massed and pure spaced lists, 20 unique words were presented twice to occupy the 40 positions. In the mixed lists 28 once-presented and six twice-presented words occupied the 40 positions. In the pure spaced lists, spacings of repeated items were chosen so that each of the lags 1-8 occurred with equal probability. In the mixed lists, massed repetitions (lag=0) and spaced repetitions (lags 1-8) were chosen such that each of the 9 lags of 0-8 were used exactly twice within each session. The order of presentation for the different list types was randomized within each session. For the first session, the first four lists were chosen so that each list type was presented exactly once. An experimenter sat in with the subject for these first four lists, though no subject had difficulty understanding the task.

The data for this experiment is stored in `data/repFR.mat`. We define a unique `prepare_repetition_data` function to build structures from the dataset that works with our existing data analysis and fitting functions.

Like in `prepare_murd_data`, we need list lengths, a data frame for visualizations with psifir, and a trials array encoding recall events as sequences of presentation positions. But we'll also need an additional array tracking presentation order, too.

In [ ]:
# export

import scipy.io as sio
import numpy as np
import pandas as pd
from psifr import fr

def prepare_repdata(path):
    """
    Prepares data formatted like `data/repFR.mat` for fitting.
    """
    
    # load all the data
    matfile = sio.loadmat(path, squeeze_me=True)['data'].item()
    subjects = matfile[0]
    pres_itemnos = matfile[4]
    recalls = matfile[6]
    list_types = matfile[7]
    list_length = matfile[12]
    
    # convert pres_itemnos into rows of unique indices for easier model encoding
    presentations = []
    for i in range(len(pres_itemnos)):
        seen = []
        presentations.append([])
        for p in pres_itemnos[i]:
            if p not in seen:
                seen.append(p)
            presentations[-1].append(seen.index(p))
    presentations = np.array(presentations)

    # discard intrusions from recalls
    trials = []
    for i in range(len(recalls)):
        trials.append([])
        
        trial = list(recalls[i])
        for t in trial:
            if (t > 0) and (t not in trials[-1]):
                trials[-1].append(t)
        
        while len(trials[-1]) < list_length:
            trials[-1].append(0)
            
    trials = np.array(trials)
    
    # encode dataset into psifr format
    data = []
    for trial_index, trial in enumerate(trials):
        presentation = presentations[trial_index]
        
        # every time the subject changes, reset list_index
        if not data or data[-1][0] != subjects[trial_index]:
            list_index = 0
        list_index += 1
        
        # add study events
        for presentation_index, presentation_event in enumerate(presentation):
            data += [[subjects[trial_index], 
                      list_index, 'study', presentation_index+1, presentation_event,  list_types[trial_index]
                     ]]
            
        # add recall events
        for recall_index, recall_event in enumerate(trial):
            if recall_event != 0:
                data += [[subjects[trial_index], list_index, 
                          'recall', recall_index+1, presentation[recall_event-1], list_types[trial_index]
                         ]]
                
    data = pd.DataFrame(data, columns=[
        'subject', 'list', 'trial_type', 'position', 'item', 'condition'])
    merged = fr.merge_free_recall(data, list_keys=['condition'])
    
    return trials, merged, list_length, presentations, list_types, data, subjects

In [ ]:
trials, events, list_length, presentations, list_types, rep_data, subjects = prepare_repdata(
    'data/repFR.mat')

events.head()

,subject,list,item,input,output,study,recall,repeat,intrusion,condition
0,1,1,0,1,1.0,True,True,0,False,4
1,1,1,1,2,2.0,True,True,0,False,4
2,1,1,2,3,3.0,True,True,0,False,4
3,1,1,3,4,4.0,True,True,0,False,4
4,1,1,4,5,5.0,True,True,0,False,4


## Simulated Datasets
The approach for creating simulated datasets is to initialize a model with specified parameters and experience sequences and then populate a psifr-formatted array with the outcomes of performing `free recall`. 

The `simulate_data` function below presumes each item is just presented once and that a model has already been initialized, and is better for quick baseline characterization of model performance. Datasets with item repetitions during presentation violate this premise; a more unique function is normally necessary for simulating these models in a performant way.

Since model simulation this way has always directly led to visualization in work done so far, a corresponding `trials` array is not produced.

In [ ]:
# export

def simulate_data(model, experiment_count, first_recall_item=None):
    """
    Initialize a model with specified parameters and experience sequences and 
    then populate a psifr-formatted dataframe with the outcomes of performing `free recall`. 
    
    **Required model attributes**:
    - item_count: specifies number of items encoded into memory
    - context: vector representing an internal contextual state
    - experience: adding a new trace to the memory model
    - free_recall: function that freely recalls a given number of items or until recall stops
    """
    
    # encode items
    try:
        model.experience(np.eye(model.item_count, model.item_count + 1, 1))
    except ValueError:
        # so we can apply to CMR
        model.experience(np.eye(model.item_count, model.item_count))

    # simulate retrieval for the specified number of times, tracking results in df
    data = []
    for experiment in range(experiment_count):
        data += [[experiment, 0, 'study', i + 1, i] for i in range(model.item_count)]
    for experiment in range(experiment_count):
        if first_recall_item is not None:
            model.force_recall(first_recall_item)
        data += [[experiment, 0, 'recall', i + 1, o] for i, o in enumerate(model.free_recall())]
    data = pd.DataFrame(data, columns=['subject', 'list', 'trial_type', 'position', 'item'])
    merged = fr.merge_free_recall(data)
    
    return merged